In [1]:
import numpy as np
import tensorflow as tf

In [2]:
DATA_DIR = "../data/"

n_input = 29
n_classes = 2
learning_rate = 0.001
num_steps = 2000
batch_size = 100
display_step = 100

In [3]:
with np.load(DATA_DIR+"ccdataset.npz") as data:
    trainset = data['train']
    features = np.float32(trainset[:,1:-1])
    labels_int = np.int32(trainset[:,-1])
    labels = np.zeros((labels_int.shape[0], n_classes))
    labels[range(labels_int.shape[0]), labels_int] = 1

In [15]:
# Assume that each row of `features` corresponds to the same row as `labels`.
assert features.shape[0] == labels.shape[0]

features_placeholder = tf.placeholder(features.dtype, features.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
dataset = dataset.batch(batch_size)
iterator = dataset.make_initializable_iterator()

sess = tf.Session()

sess.run(iterator.initializer, feed_dict={features_placeholder: features,
                                          labels_placeholder: labels})

X, Y = iterator.get_next()

In [16]:
# Create Model
def Baseline_model(x, n_classes, reuse, is_training):
    with tf.variable_scope('Baseline', reuse=reuse):
        layer1 = tf.layers.dense(inputs=x, units=50, activation=tf.nn.leaky_relu)
        layer2 = tf.layers.dense(inputs=layer1, units=50, activation=tf.nn.leaky_relu)
        layer3 = tf.layers.dense(inputs=layer2, units=50, activation=tf.nn.leaky_relu)
        out = tf.layers.dense(inputs=layer3, units=n_classes)
        out = tf.nn.softmax(out) if not is_training else out
    return out

logits_train = Baseline_model(X, n_classes, reuse=False, is_training=True)
logits_test = Baseline_model(X, n_classes, reuse=True, is_training=False)

ValueError: Variable Baseline/dense/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-5-0427b9a4f753>", line 4, in Baseline_model
    layer1 = tf.layers.dense(inputs=x, units=50, activation=tf.nn.leaky_relu)
  File "<ipython-input-5-0427b9a4f753>", line 11, in <module>
    logits_train = Baseline_model(X, n_classes, reuse=False, is_training=True)
  File "/Users/Mengxuan/tensorflow/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# Define loss and optimizer (with train logits, for dropout to take effect)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits_train, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Run the initializer
sess.run(init)

In [7]:
# Training cycle
for step in range(1, num_steps + 1):

    try:
        # Run optimization
        sess.run(train_op)
    except tf.errors.OutOfRangeError:
        # Reload the iterator when it reaches the end of the dataset
        sess.run(iterator.initializer, feed_dict={features_placeholder: features, labels_placeholder: labels})
        sess.run(train_op)

    if step % display_step == 0 or step == 1:
        # Calculate batch loss and accuracy
        # (note that this consume a new batch of data)
        loss, acc = sess.run([loss_op, accuracy])
        print("Step " + str(step) + ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))

Step 1, Minibatch Loss= 1.1372, Training Accuracy= 0.390
Step 100, Minibatch Loss= 0.1759, Training Accuracy= 0.920
Step 200, Minibatch Loss= 0.0624, Training Accuracy= 0.960
Step 300, Minibatch Loss= 0.0341, Training Accuracy= 0.990
Step 400, Minibatch Loss= 0.0435, Training Accuracy= 0.980
Step 500, Minibatch Loss= 0.0682, Training Accuracy= 0.950
Step 600, Minibatch Loss= 0.0362, Training Accuracy= 0.980
Step 700, Minibatch Loss= 0.0401, Training Accuracy= 0.980
Step 800, Minibatch Loss= 0.0427, Training Accuracy= 0.990
Step 900, Minibatch Loss= 0.0199, Training Accuracy= 0.990
Step 1000, Minibatch Loss= 0.0199, Training Accuracy= 0.990
Step 1100, Minibatch Loss= 0.0057, Training Accuracy= 1.000
Step 1200, Minibatch Loss= 0.0082, Training Accuracy= 1.000
Step 1300, Minibatch Loss= 0.0007, Training Accuracy= 1.000
Step 1400, Minibatch Loss= 0.0051, Training Accuracy= 1.000
Step 1500, Minibatch Loss= 0.0020, Training Accuracy= 1.000
Step 1600, Minibatch Loss= 0.0050, Training Accuracy